## Non Compliance Experiment=1
Test top norms for different w_nc

In [24]:
import sys
sys.path.append('../src')

In [27]:
import yaml
from IPython.utils import io
from tqdm.notebook import tqdm
from pathlib import Path
import pandas as pd
from mcmc_norm_learning.algorithm_1_v4 import to_tuple

In [2]:
def write_log(output,path):
    with open(Path(path), 'w') as f:
        f.write(output)

In [6]:
%%time
with io.capture_output() as exp_nc_0_log:
    !python ../scripts/nc_experiments.py -exp exp_nc0 -w_nc 0.0 -n_threads 6
write_log(exp_nc_0_log.stdout,"../data_nc/exp_nc0/exp_nc_0_run.log")  

CPU times: user 7.88 s, sys: 2.16 s, total: 10 s
Wall time: 28min 3s


In [7]:
!grep "Time for" ../data_nc/exp_nc0/exp_nc_0_run.log

In [8]:
!grep "log" ../data_nc/exp_nc0/exp_nc_0_run.log

In [ ]:
!dvc add ../data_nc/exp_nc0/
!git add --all
!git commit -m "Results gen for dummy_exp_nc_0"
!dvc push

### Loop over w_nc values

In [9]:
w_nc_list=[0.0,0.05,0.1,0.2,0.3,0.4]

In [10]:
for w_nc in tqdm(w_nc_list,"Loop Over diff w_nc"):
    exp_dir_name="exp_nc_1.w_nc={}".format(w_nc)
    with io.capture_output() as exp_log:
        !python ../scripts/nc_experiments.py -exp $exp_dir_name -w_nc $w_nc  -n_threads 6
    write_log(exp_log.stdout,"../data_nc/{}/run.log".format(exp_dir_name)) 

## Analysis
### 1. True Norm log_posterior for diff w_nc

In [55]:
for w_nc in w_nc_list:
    exp_dir_name="exp_nc_1.w_nc={}".format(w_nc)
    prior_out=!grep "For True Norm" ../data_nc/{exp_dir_name}/run.log
    post_out=!grep "log_post_true_norm" ../data_nc/{exp_dir_name}/run.log
    print (f"For w_nc={w_nc}:")
    print (prior_out)
    print (post_out)
    print()

For w_nc=0.0:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
['log_post_no_norm=-13732.653608350562,log_post_true_norm=-12769.387404068873']

For w_nc=0.05:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
['log_post_no_norm=-13732.653608350562,log_post_true_norm=-13097.518410107548']

For w_nc=0.1:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
['log_post_no_norm=-13732.653608350562,log_post_true_norm=-13146.65914171823']

For w_nc=0.2:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
['log_post_no_norm=-13732.653608350562,log_post_true_norm=-13187.900778087323']

For w_nc=0.3:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
['log_post_no_norm=-13732.653608350562,log_post_true_norm=-13428.957121845662']

For w_nc=0.4:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
['log_post_no_norm=-13732.653608350562,log_

### 1. True Norm atleast mentioned in any chains, for diff w_nc

In [19]:
for w_nc in w_nc_list:
    exp_dir_name="exp_nc_1.w_nc={}".format(w_nc)
    out=!grep "True norm in some chain(s)" ../data_nc/{exp_dir_name}/chain_info.txt
    print (f"For w_nc={w_nc}, True Norm found={out}")

For w_nc=0.0, True Norm found=['True norm in some chain(s): True']
For w_nc=0.05, True Norm found=['True norm in some chain(s): True']
For w_nc=0.1, True Norm found=['True norm in some chain(s): True']
For w_nc=0.2, True Norm found=['True norm in some chain(s): True']
For w_nc=0.3, True Norm found=['True norm in some chain(s): False']
For w_nc=0.4, True Norm found=['True norm in some chain(s): False']


### 2. Rank of True Norm as per log_post, iff it was found

In [28]:
with open("../params_nc.yaml", 'r') as fd:
    params = yaml.safe_load(fd)
true_norm=to_tuple(params['true_norm']['exp'])

In [52]:
for w_nc in w_nc_list:
    exp_dir_name="exp_nc_1.w_nc={}".format(w_nc)
    rank_df=pd.read_csv(f"../data_nc/{exp_dir_name}/ranked_posteriors.csv",index_col=False)
    rank_post=rank_df.loc[rank_df.expression==str(true_norm)][["post_rank","log_posterior"]].values
    print (f"For w_nc={w_nc}, True Norm rank={rank_post[0][0]},wi posterior={rank_post[0][1]}")

For w_nc=0.0, True Norm rank=1.0,wi posterior=-12783.31601471459
For w_nc=0.05, True Norm rank=1.0,wi posterior=-13111.447020753265
For w_nc=0.1, True Norm rank=1.0,wi posterior=-13160.587752363943
For w_nc=0.2, True Norm rank=1.0,wi posterior=-13201.82938873304


IndexError: index 0 is out of bounds for axis 0 with size 0

In [47]:
rank_post[0][0]

1.0

#### ==> True Norm always has highest posterior, when its found,
confirm if it is in one chain, or multiple